# Star shaped qubit

We'll be creating a 2D design and adding a star shaped qubit QComponent.


There's a central star shaped component. This component is essentially a circle with trpezoids subtracted from it. A T-junction qubit is attached to this central shape. Then there are four coupling resonators and one readout resonator present in the default setting. The shape of the central component and the number coupling resonators are customizable. Let's try out a few diferent ways to create and render this component.

In [ ]:
# So, let us dive right in. For convenience, let's begin by enabling
# automatic reloading of modules when they change.
%load_ext autoreload
%autoreload 2

Next, we import the relevant modules. You can add new modules as and when needed. 

In [ ]:
import qiskit_metal as metal
import numpy as np
from math import *
from qiskit_metal import designs, draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit, QComponent

Next we import the GUI

In [ ]:
from qiskit_metal import MetalGUI

A QDesign class must be instantiated each time a new quantum circuit design is created. The design class `DesignPlanar` is best for 2D circuit designs.

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)
gui.rebuild() 

# A Star Qubit

You can create a ready-made star qubit from the QComponent Library, qiskit_metal.qlibrary.qubits. star_qubit.py is the file containing our qubit so StarQubit is the module we import. 


In [ ]:
from qiskit_metal.qlibrary.qubits.star_qubit import StarQubit

In [ ]:
# The following default_options can be overridden by user.
StarQubit.get_template_options(design)

In [ ]:
# To force overwrite a QComponent with an existing name.  
# This is useful when re-running cells in a notebook. 
design.overwrite_enabled = True

Let us place the qubit at (x,y) =(5,5), and change the default rotation values. As this design is meant to have up to 5 contacts including one readout and up to 4 coupling resonators, these angles should be 72 degrees apart. However, the number of connectors may be changed to any value between 0 and 4.

In [ ]:
qubit_options = dict(
    pos_x = '5um',
    pos_y = '5um',  
    number_of_connectors = 4 # Change the number of connectors
)

# Create a new Concentric Transmon object with name 'Q1' 
q1 = StarQubit(design, 'Star', options=qubit_options)

gui.rebuild()  # rebuild the design and plot 
gui.autoscale() #resize GUI to see QComponent
gui.zoom_on_components(['Star']) #Can also gui.zoom_on_components([q1.name])

In [ ]:
#Let's see what the Q1 object looks like

q1 #print Q1 information

In [ ]:
#Save screenshot as a .png formatted file.
gui.screenshot()

In [ ]:
# Screenshot the canvas only as a .png formatted file.
gui.figure.savefig('shot.png')

from IPython.display import Image, display
_disp_ops = dict(width=500)
display(Image('shot.png', **_disp_ops))

# Render to Ansys

In this section we will use a semi-manual (advanced) analysis flow. Please refer to tutorial 4.2 for the `suggested` method. As illustrated, the methods are equivalent, but the advanced method allows you to directly override some renderer-specific settings.

### Finite Element Eigenmode Analysis

#### Setup

Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation.

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

The renderer can be reached from the analysis class. Let's give it a shorter alias.

In [ ]:
hfss = eig_qb.sim.renderer

Now we connect to the tool using the unified command.

In [ ]:
hfss.start()

The previous command is supposed to open ansys (if closed), create a new project and finally connect this notebook to it.

If for any reason the previous cell failed, please try the manual path described in the next three cells:
1. uncomment and execute only **one** of the lines in the first cell.
1. uncomment and execute the second cell.
1. uncomment and execute only **one** of the lines in the third cell.

In [ ]:
# hfss.open_ansys()   # this opens Ansys 2021 R2 if present
hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# hfss.open_ansys(path='C:\\Program Files\\AnsysEM\\AnsysEM21.1\\Win64')
# hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM21.1/Win64'

In [ ]:
hfss.new_ansys_project()

In [ ]:
hfss.connect_ansys()
# hfss.connect_ansys('C:\\project_path\\', 'Project1')  # will open a saved project before linking the Jupyter session

#### Render design in Ansys

Create and activate an eigenmode design called "StarQubit".

In [ ]:
hfss.activate_ansys_design("StarQubit", 'eigenmode')  # use new_ansys_design() to force creation of a blank design

Render the single qubit in Metal, called q1, to "StarQubit" design in Ansys.

In [ ]:
hfss.render_design(['Star'], [])
hfss.save_screenshot()

## Disconnect from Ansys

In [ ]:
eig_qb.sim.close()

# Closing the Qiskit Metal GUI

In [ ]:
gui.main_window.close()